<a href="https://colab.research.google.com/github/AndreaBertoglio/MLDM/blob/master/Presentazione_PreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pre-processing**


Per migliorare le prestazioni dei modelli utilizzati per l'apprendimento, è stato necessario effettuare una fase di pre-processing dei dati. In particolare, ci siamo occupati per la maggior parte di normalizzazione dei dati, gestione di dati mancanti, ricerca di eventuali outliers che interferissero con gli algoritmi di apprendimento, ed in misura minore di feature construction.

In prima battuta abbiamo fatto una rapida analisi dei dati, notando fin da subito che alcune feature del dataset presentavano distribuzioni di dati anomale, ad esempio se consideriamo la feature "density", si possono notare valori compresi tra vicini a 1 e valori vicini a 1000, mentre si nota una completa assenza di dati nei valori intermedi. Questo è stato probabilmente causato dall'utilizzo di unità di misura diverse per record diversi (verosimilmente mg/l e g/l nel caso di esempio). Un discorso simile vale anche per la "volatile acidity", anch'essa presenta valori disomogenei probabilmente dovuti a diverse unità di misura. Di conseguenza, per prima cosa sono state uniformate le unità di misura di ogni singola feature.

Come secondo fatto, si è notato che le varie feature tra loro, avevano valori molto diversi dato che misurando grandezze differenti si usano misure differenti (ad esempio per densità e pH), e quindi difficili da confrontare. Quindi è stato necessario normalizzare i dati per consentirne una migliore valutazione. Per la normalizzazione si sono visti due approcci distinti, uno che prevede la riscalatura dei valori su un range compreso tra 0 e 1, e un secondo che riscala i dati su una distribuzione normale con media 0 e varianza 1. I risultati migliori sono stati ottenuti con il secondo approccio, quindi d'ora in avanti si considererà solo quello.

Anche solo con questi due accorgimenti in fase di pre-processing le prestazioni dei modelli addestrati sono migliorate sensibilmente.




Inoltre, dall'analisi dei dati è risultato che anche dopo la normalizzazione alcuni valori risultavano "fuori scala", ovvero risultavano molto distanti dalla maggior parte degli altri valori. Per ovviare al problema è stato deciso di rimuovere quei valori che risultavano fuori scala e gestirli come missing values.
Il punto cruciale di questa gestione è stato stabilire la giusta soglia per cui considerare un valore fuori scala oppure no. Per far ciò abbiamo analizzato i valori su diverse soglie.

Grafico 1: valori fuori scala totali
![GRAFICO1](https://raw.githubusercontent.com/AndreaBertoglio/MLDM/master/Immagini%20Presentazione/Grafico%20percentuale%20valori%20fuori%20scala%20totali.PNG)

Grafico 2: valori fuori scala
![GRAFICO2](https://raw.githubusercontent.com/AndreaBertoglio/MLDM/master/Immagini%20Presentazione/Grafico%20valori%20fuori%20scala%20per%20ogni%20feature.PNG)

Grafico 3: particolare del grafico 2
![GRAFICO3](https://raw.githubusercontent.com/AndreaBertoglio/MLDM/master/Immagini%20Presentazione/Grafico%20valori%20fuori%20scala%20per%20ogni%20feature%20(zoom).PNG)

Da queste analisi abbiamo ritenuto opportuno non utilizzare soglie troppo basse, poiché in questo modo avremmo potuto modificare anche dati corretti, e notando che con soglie superiori a 3.5 il numero di valori fuori scala non diminuiva significativamente, abbiamo ritenuto opportuno creare 3 dataset differenti con soglia rispettivamente a 3, 3.5 e 4. I migliori risultati si sono ottenuti comunque con la soglia di 3.5
Come detto in precedenza i dati “fuori scala” sono stati cancellati e considerati missing values.


Per la gestione dei missing values è stato dapprima utilizzato una semplice strategia di gestione sostituendo ogni valore mancante con la media degli altri valori, ed in seguito altre strategie abbastanza semplici, come il valore più frequente. Tuttavia, tali strategie risultavano poco efficaci in quanto generavano dati non veritieri o non rilevanti, quindi siamo passati ad un approccio leggermente più complesso con un imputer iterativo, che modella ogni feature con missing values in funzione di altre feature. Il processo avviene in modo iterativo: ad ogni passo, una colonna delle feature è designata come output y e le altre colonne sono trattate come input X. Viene creato un regressore sulla base di (X, y) per le y conosciute. Poi, il regressore viene usato per predire i missing values di y. Questo viene fatto per ogni caratteristica in modo iterativo, e poi è ripetuto più volte per avere una maggiore accuratezza.

Successivamente abbiamo provato a evidenziare quei dati che risultavano particolarmente diversi degli altri della stessa classe, ad esempio un elemento di classe “good”, ma che era molto distante dagli altri elementi della stessa classe e più simile a quelli di classe “bad”.
Per fare ciò abbiamo implementato un algoritmo per il riconoscimento di questi elementi, gli outliers. Tale algoritmo è basato sulla distanza di ogni elemento dagli altri elementi della stessa classe, in particolare si calcola la media delle distanze di ogni elemento da tutti gli altri. La distanza utilizzata è stata la distanza euclidea, e per la decisione della soglia è stato fatto un ragionamento simile a quello per la gestione dei valori fuori scala.

Grafico 4: Outliers detection
![GRAFICO4](https://raw.githubusercontent.com/AndreaBertoglio/MLDM/master/Immagini%20Presentazione/Grafico%20outliers%20a%20confronto%201%25.png)

Nella valutazione per la scelta della soglia abbiamo anche considerato l’ordine di esecuzione delle varie fasi di pre-processing. Ovviamente per prima avviene la normalizzazione dei dati e la gestione dei missing values, ma l’identificazione di outliers e la gestione dei valori fuori scala poteva essere arbitraria.
Dai risultati ottenuti si può notare come la gestione di valori fuori scala riduca notevolmente gli outliers identificati, di conseguenza abbiamo deciso di effettuare prima la gestione dei fuori scala e successivamente quella degli outliers.
Per la decisione sulla soglia si nota come con soglie inferiori a 20 tutti i dati sono considerati outliers, mentre la riduzione più consistente si ha fino a un valore di 50. Quindi, non volendo eliminare una parte troppo consistente del dataset, abbiamo deciso di eliminare fino ad un massimo dell'1% dei dati e anche in questo caso abbiamo creato diversi dataset con soglie di 40, 50 e 60.
Una volta identificati gli elementi outliers, la loro nostra decisione è stata semplicemente quella di non considerare tali elementi nei nostri algoritmi di apprendimento e quindi di eliminarli dal dataset.
Alla luce di ciò le prestazioni migliori sono state ottenute con la soglia pari a 40, ovvero considerando outliers tutti gli elementi con distanza media dagli altri superiore a 40.
Con queste operazioni aggiuntive le prestazioni dei modelli hanno subito un leggero miglioramento, ovviamente meno significativo per i modelli che per loro natura sono meno sensibili agli outliers (ad esempio alberi di decisione e random forest), e più marcati in modelli più sensibili come le Support Vector Machine.


Infine, abbiamo provato un approccio di feature construction, in particolare utilizzando un semplice algoritmo di clustering e utilizzando la suddivisione in cluster come feature aggiuntiva. L’algoritmo scelto è stato il K-means, con 10 cluster.
Tuttavia, non abbiamo riscontrato significativi miglioramenti, salvo qualche caso in alberi di decisione e random forest.
In alternativa abbiamo tentato un approccio basato sull’analisi delle feature disponibili, notando come “fixed acidity” sia fortemente correlata con “citric acid” e “density”. Di conseguenza abbiamo pensato di utilizzare due nuove feature che fossero combinazione delle precedenti:

“citric acidity” = “fixed acidity” * “citric acid”

“density acidity” = “fixed acidity” * “density”

Anche questo approccio però si è rivelato poco efficace e quindi non è stato approffondito e utilizzato.
